In [3]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
spark.range(5).show()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/20 13:44:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/20 13:44:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



In [4]:
import multiprocessing
# import pyspark

nprocs = multiprocessing.cpu_count()

spark = (pyspark.sql.SparkSession.builder
 .master('local')
 .config('spark.jars.packages', 'mysql:mysql-connector-java:8.0.16')
 .config('spark.driver.memory', '4G')
 .config('spark.driver.cores', nprocs)
 .config('spark.sql.shuffle.partitions', nprocs)
 .appName('MySparkApplication')
 .getOrCreate())


22/10/20 13:44:37 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
import pyspark.sql.functions as F

In [6]:
import pandas as pd
import numpy as np

np.random.seed(456)

pandas_dataframe = pd.DataFrame(
    dict(n=np.arange(20), group=np.random.choice(list("abc"), 20))
)
pandas_dataframe


,n,group
0,0,b
1,1,b
2,2,c
3,3,a
4,4,c
5,5,c
6,6,a
7,7,b
8,8,a
9,9,b


In [7]:
df = spark.createDataFrame(pandas_dataframe)
df

DataFrame[n: bigint, group: string]

In [8]:
df.show(5)

+---+-----+
|  n|group|
+---+-----+
|  0|    b|
|  1|    b|
|  2|    c|
|  3|    a|
|  4|    c|
+---+-----+
only showing top 5 rows



In [9]:
df.describe()

DataFrame[summary: string, n: string, group: string]

In [10]:
df.describe().show()

+-------+-----------------+-----+
|summary|                n|group|
+-------+-----------------+-----+
|  count|               20|   20|
|   mean|              9.5| null|
| stddev|5.916079783099616| null|
|    min|                0|    a|
|    max|               19|    c|
+-------+-----------------+-----+



In [11]:
from IPython.display import display, Markdown

In [12]:
display(Markdown(df.describe().show()))

+-------+-----------------+-----+
|summary|                n|group|
+-------+-----------------+-----+
|  count|               20|   20|
|   mean|              9.5| null|
| stddev|5.916079783099616| null|
|    min|                0|    a|
|    max|               19|    c|
+-------+-----------------+-----+



<IPython.core.display.Markdown object>

In [13]:
print(df.describe().show())

+-------+-----------------+-----+
|summary|                n|group|
+-------+-----------------+-----+
|  count|               20|   20|
|   mean|              9.5| null|
| stddev|5.916079783099616| null|
|    min|                0|    a|
|    max|               19|    c|
+-------+-----------------+-----+

None


|-------|-----------------|-----|
|summary|                n|group|
|-------|-----------------|-----|
|  count|               20|   20|
|   mean|              9.5| null|
| stddev|5.916079783099616| null|
|    min|                0|    a|
|    max|               19|    c|


In [14]:
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)


+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [15]:
mpg.hwy

Column<'hwy'>

In [16]:
mpg.select(mpg.hwy, mpg.cty, mpg.model).show(10)

+---+---+----------+
|hwy|cty|     model|
+---+---+----------+
| 29| 18|        a4|
| 29| 21|        a4|
| 31| 20|        a4|
| 30| 21|        a4|
| 26| 16|        a4|
| 26| 18|        a4|
| 27| 18|        a4|
| 26| 18|a4 quattro|
| 25| 16|a4 quattro|
| 28| 20|a4 quattro|
+---+---+----------+
only showing top 10 rows



In [17]:
mpg.select(mpg.hwy, mpg.cty, mpg.model).show()

+---+---+------------------+
|hwy|cty|             model|
+---+---+------------------+
| 29| 18|                a4|
| 29| 21|                a4|
| 31| 20|                a4|
| 30| 21|                a4|
| 26| 16|                a4|
| 26| 18|                a4|
| 27| 18|                a4|
| 26| 18|        a4 quattro|
| 25| 16|        a4 quattro|
| 28| 20|        a4 quattro|
| 27| 19|        a4 quattro|
| 25| 15|        a4 quattro|
| 25| 17|        a4 quattro|
| 25| 17|        a4 quattro|
| 25| 15|        a4 quattro|
| 24| 15|        a6 quattro|
| 25| 17|        a6 quattro|
| 23| 16|        a6 quattro|
| 20| 14|c1500 suburban 2wd|
| 15| 11|c1500 suburban 2wd|
+---+---+------------------+
only showing top 20 rows



In [18]:
mpg.hwy + 1

Column<'(hwy + 1)'>

In [19]:
mpg.select(mpg.hwy, mpg.hwy + 1).show(10)

+---+---------+
|hwy|(hwy + 1)|
+---+---------+
| 29|       30|
| 29|       30|
| 31|       32|
| 30|       31|
| 26|       27|
| 26|       27|
| 27|       28|
| 26|       27|
| 25|       26|
| 28|       29|
+---+---------+
only showing top 10 rows



In [20]:
col1 = mpg.hwy.alias('highway_mileage')
col2 = (mpg.hwy / 2).alias('half_highway_milage')

mpg.select(col1, col2).show(5)

+---------------+-------------------+
|highway_mileage|half_highway_milage|
+---------------+-------------------+
|             29|               14.5|
|             29|               14.5|
|             31|               15.5|
|             30|               15.0|
|             26|               13.0|
+---------------+-------------------+
only showing top 5 rows



In [21]:
from pyspark.sql.functions import col, expr


In [22]:
col('hwy')

Column<'hwy'>

In [23]:
avg_column = (col('hwy') + col('cty')) / 2


In [24]:
mpg.select(col('hwy').alias('highway_milage'),
           mpg.cty.alias('city_milage'),
           avg_column.alias('avg_milage')).show(10)

+--------------+-----------+----------+
|highway_milage|city_milage|avg_milage|
+--------------+-----------+----------+
|            29|         18|      23.5|
|            29|         21|      25.0|
|            31|         20|      25.5|
|            30|         21|      25.5|
|            26|         16|      21.0|
|            26|         18|      22.0|
|            27|         18|      22.5|
|            26|         18|      22.0|
|            25|         16|      20.5|
|            28|         20|      24.0|
+--------------+-----------+----------+
only showing top 10 rows



In [25]:
mpg.select(
    expr('hwy'),
    expr('hwy + 1'),
    expr('hwy AS Highway_Mileage'),
    expr('hwy + 1 AS highway_incremented'),
).show(10)

+---+---------+---------------+-------------------+
|hwy|(hwy + 1)|Highway_Mileage|highway_incremented|
+---+---------+---------------+-------------------+
| 29|       30|             29|                 30|
| 29|       30|             29|                 30|
| 31|       32|             31|                 32|
| 30|       31|             30|                 31|
| 26|       27|             26|                 27|
| 26|       27|             26|                 27|
| 27|       28|             27|                 28|
| 26|       27|             26|                 27|
| 25|       26|             25|                 26|
| 28|       29|             28|                 29|
+---+---------+---------------+-------------------+
only showing top 10 rows



In [26]:
mpg.select(
    mpg.hwy.alias("highway"),
    col("hwy").alias("highway"),
    expr("hwy").alias("highway"),
    expr("hwy AS highway"),
).show(5)


+-------+-------+-------+-------+
|highway|highway|highway|highway|
+-------+-------+-------+-------+
|     29|     29|     29|     29|
|     29|     29|     29|     29|
|     31|     31|     31|     31|
|     30|     30|     30|     30|
|     26|     26|     26|     26|
+-------+-------+-------+-------+
only showing top 5 rows



In [27]:
mpg.createOrReplaceTempView("mpg")

In [28]:
spark.sql('''
SELECT hwy, cty, (hwy + cty) / 2 AS avg
FROM mpg
''')

DataFrame[hwy: bigint, cty: bigint, avg: double]

In [29]:
spark.sql('''
SELECT hwy, cty, (hwy + cty) / 2 AS avg
FROM mpg
LIMIT 10
''').show(20)

+---+---+----+
|hwy|cty| avg|
+---+---+----+
| 29| 18|23.5|
| 29| 21|25.0|
| 31| 20|25.5|
| 30| 21|25.5|
| 26| 16|21.0|
| 26| 18|22.0|
| 27| 18|22.5|
| 26| 18|22.0|
| 25| 16|20.5|
| 28| 20|24.0|
+---+---+----+



In [30]:
mpg.dtypes

[('manufacturer', 'string'),
 ('model', 'string'),
 ('displ', 'double'),
 ('year', 'bigint'),
 ('cyl', 'bigint'),
 ('trans', 'string'),
 ('drv', 'string'),
 ('cty', 'bigint'),
 ('hwy', 'bigint'),
 ('fl', 'string'),
 ('class', 'string')]

In [31]:
mpg.printSchema()

root
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- displ: double (nullable = true)
 |-- year: long (nullable = true)
 |-- cyl: long (nullable = true)
 |-- trans: string (nullable = true)
 |-- drv: string (nullable = true)
 |-- cty: long (nullable = true)
 |-- hwy: long (nullable = true)
 |-- fl: string (nullable = true)
 |-- class: string (nullable = true)



In [32]:
mpg.select(mpg.hwy.cast("string")).printSchema()

root
 |-- hwy: string (nullable = true)



In [33]:
n = mpg.select(mpg.hwy.cast("string")).printSchema()

root
 |-- hwy: string (nullable = true)



In [34]:
print(n)

None


In [35]:
type(n)

NoneType

In [36]:
n = mpg.select(mpg.hwy.cast('string'))

In [37]:
n

DataFrame[hwy: string]

In [38]:
n.printSchema()

root
 |-- hwy: string (nullable = true)



In [39]:
n.show(5)

+---+
|hwy|
+---+
| 29|
| 29|
| 31|
| 30|
| 26|
+---+
only showing top 5 rows



In [41]:
# from pyspark.sql.functions import concat, sum, 
import pyspark.sql.functions as F
from pyspark.sql.functions import * 


In [42]:
F.sum

<function pyspark.sql.functions.sum(col: 'ColumnOrName') -> pyspark.sql.column.Column>

In [43]:
mpg.select((F.sum(mpg.hwy) / F.count(mpg.hwy)).alias('average'),
           F.avg(expr('hwy AS avg2'))).show()

+-----------------+-----------------+
|          average| avg(hwy AS avg2)|
+-----------------+-----------------+
|23.44017094017094|23.44017094017094|
+-----------------+-----------------+



In [44]:
mpg.select(F.concat(mpg.cyl, F.lit(' cylinders'))).show()

+-----------------------+
|concat(cyl,  cylinders)|
+-----------------------+
|            4 cylinders|
|            4 cylinders|
|            4 cylinders|
|            4 cylinders|
|            6 cylinders|
|            6 cylinders|
|            6 cylinders|
|            4 cylinders|
|            4 cylinders|
|            4 cylinders|
|            4 cylinders|
|            6 cylinders|
|            6 cylinders|
|            6 cylinders|
|            6 cylinders|
|            6 cylinders|
|            6 cylinders|
|            8 cylinders|
|            8 cylinders|
|            8 cylinders|
+-----------------------+
only showing top 20 rows



In [45]:
mpg.select(F.concat(mpg.cyl, F.lit(' cylinders')).alias('Cylinders')).show()

+-----------+
|  Cylinders|
+-----------+
|4 cylinders|
|4 cylinders|
|4 cylinders|
|4 cylinders|
|6 cylinders|
|6 cylinders|
|6 cylinders|
|4 cylinders|
|4 cylinders|
|4 cylinders|
|4 cylinders|
|6 cylinders|
|6 cylinders|
|6 cylinders|
|6 cylinders|
|6 cylinders|
|6 cylinders|
|8 cylinders|
|8 cylinders|
|8 cylinders|
+-----------+
only showing top 20 rows



In [46]:
from pyspark.sql.functions import regexp_extract, regexp_replace


In [47]:
textdf = spark.createDataFrame(
    pd.DataFrame(
        {
            "address": [
                "600 Navarro St ste 600, San Antonio, TX 78205",
                "3130 Broadway St, San Antonio, TX 78209",
                "303 Pearl Pkwy, San Antonio, TX 78215",
                "1255 SW Loop 410, San Antonio, TX 78227",
            ]
        }
    )
)

textdf.show(truncate=False)


+---------------------------------------------+
|address                                      |
+---------------------------------------------+
|600 Navarro St ste 600, San Antonio, TX 78205|
|3130 Broadway St, San Antonio, TX 78209      |
|303 Pearl Pkwy, San Antonio, TX 78215        |
|1255 SW Loop 410, San Antonio, TX 78227      |
+---------------------------------------------+



<b>The `regexp_extract` function lets us specify a regular expression with at least one capture group, and create a new column based on the contents of a capture group.</b>

In [48]:
textdf.select(
    "address",
    regexp_extract("address", r"^(\d+)", 1).alias("street_no"),
    regexp_extract("address", r"^\d+\s([\w\s]+?),", 1).alias("street"),
).show(truncate=False)


+---------------------------------------------+---------+------------------+
|address                                      |street_no|street            |
+---------------------------------------------+---------+------------------+
|600 Navarro St ste 600, San Antonio, TX 78205|600      |Navarro St ste 600|
|3130 Broadway St, San Antonio, TX 78209      |3130     |Broadway St       |
|303 Pearl Pkwy, San Antonio, TX 78215        |303      |Pearl Pkwy        |
|1255 SW Loop 410, San Antonio, TX 78227      |1255     |SW Loop 410       |
+---------------------------------------------+---------+------------------+



**In the example above, the first argument to `regexp_extract` is the name of the string column to extract from, the second argument is the regular expression itself, and the last argument specifies which capture group we want to use. If, for example, our regular expression had 2 capture groups in it and we wanted the contents of the 2nd group, we would specify a 2 here.**

**In addition to `regexp_extract`, `regexp_replace` lets us make substitutions based on a regular expression.**

In [49]:
textdf.select(
    "address",
    regexp_replace("address", r"^.*?,\s*", "").alias("city_state_zip"),
).show(truncate=False)


+---------------------------------------------+---------------------+
|address                                      |city_state_zip       |
+---------------------------------------------+---------------------+
|600 Navarro St ste 600, San Antonio, TX 78205|San Antonio, TX 78205|
|3130 Broadway St, San Antonio, TX 78209      |San Antonio, TX 78209|
|303 Pearl Pkwy, San Antonio, TX 78215        |San Antonio, TX 78215|
|1255 SW Loop 410, San Antonio, TX 78227      |San Antonio, TX 78227|
+---------------------------------------------+---------------------+



## .filter and .where
Spark provides two dataframe methods, `.filter` and `.where`, which both allow us to select a subset of the rows of our dataframe.

In [50]:
mpg.filter(mpg.cyl == 4).where(mpg["class"] == "subcompact").show()

+------------+-----------+-----+----+---+----------+---+---+---+---+----------+
|manufacturer|      model|displ|year|cyl|     trans|drv|cty|hwy| fl|     class|
+------------+-----------+-----+----+---+----------+---+---+---+---+----------+
|       honda|      civic|  1.6|1999|  4|manual(m5)|  f| 28| 33|  r|subcompact|
|       honda|      civic|  1.6|1999|  4|  auto(l4)|  f| 24| 32|  r|subcompact|
|       honda|      civic|  1.6|1999|  4|manual(m5)|  f| 25| 32|  r|subcompact|
|       honda|      civic|  1.6|1999|  4|manual(m5)|  f| 23| 29|  p|subcompact|
|       honda|      civic|  1.6|1999|  4|  auto(l4)|  f| 24| 32|  r|subcompact|
|       honda|      civic|  1.8|2008|  4|manual(m5)|  f| 26| 34|  r|subcompact|
|       honda|      civic|  1.8|2008|  4|  auto(l5)|  f| 25| 36|  r|subcompact|
|       honda|      civic|  1.8|2008|  4|  auto(l5)|  f| 24| 36|  c|subcompact|
|       honda|      civic|  2.0|2008|  4|manual(m6)|  f| 21| 29|  p|subcompact|
|     hyundai|    tiburon|  2.0|1999|  4

## When and Otherwise
Similar to an `IF` in Excel, `CASE...WHEN` in SQL, or `np.where` in python, spark provides a `when` function.

The when function lets us specify a condition, and a value to produce if that condition is true:

In [51]:
from pyspark.sql.functions import when

In [52]:
mpg.select(mpg.hwy, when(mpg.hwy > 25, "good_mileage").alias("mpg_desc")).show(
    12
)

+---+------------+
|hwy|    mpg_desc|
+---+------------+
| 29|good_mileage|
| 29|good_mileage|
| 31|good_mileage|
| 30|good_mileage|
| 26|good_mileage|
| 26|good_mileage|
| 27|good_mileage|
| 26|good_mileage|
| 25|        null|
| 28|good_mileage|
| 27|good_mileage|
| 25|        null|
+---+------------+
only showing top 12 rows



Notice here that if the condition we specified is false, `null` will be produced. Instead of null, we can specify a value to use if our condition is false with the `.otherwise` method.

In [53]:
mpg.select(
    mpg.hwy,
    when(mpg.hwy > 25, "good_mileage")
    .otherwise("bad_mileage")
    .alias("mpg_desc"),
).show(12)

+---+------------+
|hwy|    mpg_desc|
+---+------------+
| 29|good_mileage|
| 29|good_mileage|
| 31|good_mileage|
| 30|good_mileage|
| 26|good_mileage|
| 26|good_mileage|
| 27|good_mileage|
| 26|good_mileage|
| 25| bad_mileage|
| 28|good_mileage|
| 27|good_mileage|
| 25| bad_mileage|
+---+------------+
only showing top 12 rows



To specify multiple conditions, we can chain `.when` calls. The first condition that is met will be the value that is used, and if none of the conditions are met the value specified in the `.otherwise` will be used (or `null` if you don't provide a `.otherwise`).

In [54]:
mpg.select(
    mpg.displ,
    (
        when(mpg.displ < 2, "small")
        .when(mpg.displ < 3, "medium")
        .otherwise("large")
        .alias("engine_size")
    ),
).show(10)


+-----+-----------+
|displ|engine_size|
+-----+-----------+
|  1.8|      small|
|  1.8|      small|
|  2.0|     medium|
|  2.0|     medium|
|  2.8|     medium|
|  2.8|     medium|
|  3.1|      large|
|  1.8|      small|
|  1.8|      small|
|  2.0|     medium|
+-----+-----------+
only showing top 10 rows



Notice here that a car with a `displ` of 1.8 matches both conditions we specified, but `small` is produced because it is associated with the first matching condition. For any value between 2 and 3, `medium` will be produced, and anything larger than 3 will produce `large`.

## Sorting and Ordering
Spark lets us sort the rows in our dataframe by one or multiple columns with two methods: `.sort`, and `.orderBy`. `.sort` and `.orderBy` are aliases of each other and do the exact same thing. Like other methods we've seen, `.sort` takes in a Column object or a string that is the name of a column.

In [55]:
mpg.sort(mpg.hwy).show(8)

+------------+-------------------+-----+----+---+----------+---+---+---+---+------+
|manufacturer|              model|displ|year|cyl|     trans|drv|cty|hwy| fl| class|
+------------+-------------------+-----+----+---+----------+---+---+---+---+------+
|       dodge|ram 1500 pickup 4wd|  4.7|2008|  8|manual(m6)|  4|  9| 12|  e|pickup|
|       dodge|ram 1500 pickup 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|pickup|
|       dodge|        durango 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|   suv|
|       dodge|  dakota pickup 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|pickup|
|        jeep| grand cherokee 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|   suv|
|   chevrolet|    k1500 tahoe 4wd|  5.3|2008|  8|  auto(l4)|  4| 11| 14|  e|   suv|
|        jeep| grand cherokee 4wd|  6.1|2008|  8|  auto(l5)|  4| 11| 14|  p|   suv|
|   chevrolet|    k1500 tahoe 4wd|  5.7|1999|  8|  auto(l4)|  4| 11| 15|  r|   suv|
+------------+-------------------+-----+----+---+----------+---+---+---+---+

By default, values are sorted in ascending order. To sort in descending order, we can use the `.desc` method on any Column object, or the desc function from `pyspark.sql.functions`.

In [56]:
from pyspark.sql.functions import asc, desc


In [57]:
mpg.sort(mpg.hwy.desc())
# is the same as
mpg.sort(col("hwy").desc())
# is the same as
mpg.sort(desc("hwy")).show(5)


+------------+----------+-----+----+---+----------+---+---+---+---+----------+
|manufacturer|     model|displ|year|cyl|     trans|drv|cty|hwy| fl|     class|
+------------+----------+-----+----+---+----------+---+---+---+---+----------+
|  volkswagen|new beetle|  1.9|1999|  4|manual(m5)|  f| 35| 44|  d|subcompact|
|  volkswagen|     jetta|  1.9|1999|  4|manual(m5)|  f| 33| 44|  d|   compact|
|  volkswagen|new beetle|  1.9|1999|  4|  auto(l4)|  f| 29| 41|  d|subcompact|
|      toyota|   corolla|  1.8|2008|  4|manual(m5)|  f| 28| 37|  r|   compact|
|       honda|     civic|  1.8|2008|  4|  auto(l5)|  f| 24| 36|  c|subcompact|
+------------+----------+-----+----+---+----------+---+---+---+---+----------+
only showing top 5 rows



To specify sorting by multiple columns, we provide each column as a separate argument to .sort.

In [58]:
mpg.sort(desc("class"), mpg.cyl.asc(), col("hwy").desc()).show()


+------------+------------------+-----+----+---+----------+---+---+---+---+-----+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-----+
|      subaru|      forester awd|  2.5|2008|  4|manual(m5)|  4| 20| 27|  r|  suv|
|      subaru|      forester awd|  2.5|2008|  4|  auto(l4)|  4| 20| 26|  r|  suv|
|      subaru|      forester awd|  2.5|1999|  4|manual(m5)|  4| 18| 25|  r|  suv|
|      subaru|      forester awd|  2.5|2008|  4|manual(m5)|  4| 19| 25|  p|  suv|
|      subaru|      forester awd|  2.5|1999|  4|  auto(l4)|  4| 18| 24|  r|  suv|
|      subaru|      forester awd|  2.5|2008|  4|  auto(l4)|  4| 18| 23|  p|  suv|
|      toyota|       4runner 4wd|  2.7|1999|  4|manual(m5)|  4| 15| 20|  r|  suv|
|      toyota|       4runner 4wd|  2.7|1999|  4|  auto(l4)|  4| 16| 20|  r|  suv|
|        jeep|grand cherokee 4wd|  3.0|2008|  6|  auto(l5)|  4| 17| 22|  d|  suv|
|        jeep|gr

Grouping and Aggregating
To aggregate our data by group, we can use the .groupBy method. Like with .select, we can pass either Column objects or strings that are column names to .groupBy. All of the expressions below are equivalent.

In [59]:
mpg.groupBy(mpg.cyl)
mpg.groupBy(col("cyl"))
mpg.groupBy("cyl")


In [60]:
mpg.groupBy(mpg.cyl).agg(avg(mpg.cty), avg(mpg.hwy)).show()


+---+------------------+-----------------+
|cyl|          avg(cty)|         avg(hwy)|
+---+------------------+-----------------+
|  4|21.012345679012345|28.80246913580247|
|  8|12.571428571428571|17.62857142857143|
|  6| 16.21518987341772|22.82278481012658|
|  5|              20.5|            28.75|
+---+------------------+-----------------+



In [61]:
mpg.rollup("cyl").count().sort("cyl").show()


+----+-----+
| cyl|count|
+----+-----+
|null|  234|
|   4|   81|
|   5|    4|
|   6|   79|
|   8|   70|
+----+-----+



In [62]:
mpg.rollup("cyl").agg(expr("avg(hwy)")).sort("cyl").show()


+----+-----------------+
| cyl|         avg(hwy)|
+----+-----------------+
|null|23.44017094017094|
|   4|28.80246913580247|
|   5|            28.75|
|   6|22.82278481012658|
|   8|17.62857142857143|
+----+-----------------+



In [63]:
mpg.rollup("cyl", "class").mean("hwy").sort(col("cyl"), col("class")).show()

+----+----------+------------------+
| cyl|     class|          avg(hwy)|
+----+----------+------------------+
|null|      null| 23.44017094017094|
|   4|      null| 28.80246913580247|
|   4|   compact|          29.46875|
|   4|   midsize|           29.1875|
|   4|   minivan|              24.0|
|   4|    pickup|20.666666666666668|
|   4|subcompact| 30.80952380952381|
|   4|       suv|             23.75|
|   5|      null|             28.75|
|   5|   compact|              29.0|
|   5|subcompact|              28.5|
|   6|      null| 22.82278481012658|
|   6|   compact|25.307692307692307|
|   6|   midsize| 26.26086956521739|
|   6|   minivan|              22.2|
|   6|    pickup|              17.9|
|   6|subcompact|24.714285714285715|
|   6|       suv|              18.5|
|   8|      null| 17.62857142857143|
|   8|   2seater|              24.8|
+----+----------+------------------+
only showing top 20 rows



## Crosstabs and Pivot Tables
In addition to groupby, spark provides a couple other ways to do aggregation. One of which is .crosstab. This is very similary to pandas .crosstab function, in that it calculates the number of occurances of each unique value from the two passed columns:

In [64]:
mpg.crosstab("class", "cyl").show()

+----------+---+---+---+---+
| class_cyl|  4|  5|  6|  8|
+----------+---+---+---+---+
|   midsize| 16|  0| 23|  2|
|subcompact| 21|  2|  7|  5|
|   2seater|  0|  0|  0|  5|
|    pickup|  3|  0| 10| 20|
|   minivan|  1|  0| 10|  0|
|       suv|  8|  0| 16| 38|
|   compact| 32|  2| 13|  0|
+----------+---+---+---+---+



.crosstab simply does counts, if we want a different aggregation, we can use .pivot. For example, to find the average highway mileage for each combination of car class and number of cylinders, we could write the following:

In [65]:
mpg.groupby("class").pivot("cyl").mean("hwy").show()

+----------+------------------+----+------------------+------------------+
|     class|                 4|   5|                 6|                 8|
+----------+------------------+----+------------------+------------------+
|   midsize|           29.1875|null| 26.26086956521739|              24.0|
|       suv|             23.75|null|              18.5|16.789473684210527|
|    pickup|20.666666666666668|null|              17.9|              15.8|
|   minivan|              24.0|null|              22.2|              null|
|subcompact| 30.80952380952381|28.5|24.714285714285715|              21.6|
|   2seater|              null|null|              null|              24.8|
|   compact|          29.46875|29.0|25.307692307692307|              null|
+----------+------------------+----+------------------+------------------+



Here the unique values from the column we group by will be the rows in the resulting dataframe, and the unique values from the column we pivot on will become the columns. The values in each cell will be equal to the aggregation we specified over the group of values defined by the intersection of the rows and the columns.

## Handling Missing Data
Let's take a look at how spark handles missing data. First we'll create a dataframe that has a few missing values:

In [66]:
df = spark.createDataFrame(
    pd.DataFrame(
        {"x": [1, 2, np.nan, 4, 5, np.nan], "y": [np.nan, 0, 0, 3, 1, np.nan]}
    )
)
df.show()

+---+---+
|  x|  y|
+---+---+
|1.0|NaN|
|2.0|0.0|
|NaN|0.0|
|4.0|3.0|
|5.0|1.0|
|NaN|NaN|
+---+---+



Spark provides two main ways to deal with missing values:

.fill: to replace missing values with a specified value
.drop: to drop rows containing missing values
Both methods are accessed through the .na property. We'll look at some examples below:

In [67]:
df.na.drop().show()

+---+---+
|  x|  y|
+---+---+
|2.0|0.0|
|4.0|3.0|
|5.0|1.0|
+---+---+



In [68]:
df.na.fill(0).show()

+---+---+
|  x|  y|
+---+---+
|1.0|0.0|
|2.0|0.0|
|0.0|0.0|
|4.0|3.0|
|5.0|1.0|
|0.0|0.0|
+---+---+



For both methods, we can specify that we only want to fill or drop values in a specific column with a second argument:

In [69]:
df.na.fill(0, subset="x").show()

+---+---+
|  x|  y|
+---+---+
|1.0|NaN|
|2.0|0.0|
|0.0|0.0|
|4.0|3.0|
|5.0|1.0|
|0.0|NaN|
+---+---+



Notice that above the na values in the x column were filled with 0, but the na values in y were left alone.

In [70]:
df.na.drop(subset="y").show()

+---+---+
|  x|  y|
+---+---+
|2.0|0.0|
|NaN|0.0|
|4.0|3.0|
|5.0|1.0|
+---+---+



In the example above, the rows that had an na value for the y column were dropped, but the rows with na values for only the x column are still present.

# Explaining DataFrame Transformations
The .explain method will show us how spark is thinking about our dataframe.

In [71]:
mpg.explain()

== Physical Plan ==
*(1) Scan ExistingRDD[manufacturer#843,model#844,displ#845,year#846L,cyl#847L,trans#848,drv#849,cty#850L,hwy#851L,fl#852,class#853]




In [72]:
mpg.select(mpg.cyl, mpg.hwy).explain()

== Physical Plan ==
*(1) Project [cyl#847L, hwy#851L]
+- *(1) Scan ExistingRDD[manufacturer#843,model#844,displ#845,year#846L,cyl#847L,trans#848,drv#849,cty#850L,hwy#851L,fl#852,class#853]




In [73]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



In [74]:
print(weather.count(), "rows", len(weather.columns), "columns")

1461 rows 6 columns


In [75]:
min_date, max_date = weather.select(min("date"), max("date")).first()
min_date, max_date


('2012-01-01', '2015-12-31')

In [76]:
weather = weather.withColumn(
    "temp_avg", expr("ROUND(temp_min + temp_max) / 2")
).drop("temp_max", "temp_min")
weather.show(6)

+----------+-------------+----+-------+--------+
|      date|precipitation|wind|weather|temp_avg|
+----------+-------------+----+-------+--------+
|2012-01-01|          0.0| 4.7|drizzle|     9.0|
|2012-01-02|         10.9| 4.5|   rain|     6.5|
|2012-01-03|          0.8| 2.3|   rain|     9.5|
|2012-01-04|         20.3| 4.7|   rain|     9.0|
|2012-01-05|          1.3| 6.1|   rain|     6.0|
|2012-01-06|          2.5| 2.2|   rain|     3.5|
+----------+-------------+----+-------+--------+
only showing top 6 rows



In [77]:
from pyspark.sql.functions import month, year, quarter

In [78]:
(
    weather.withColumn("month", month("date"))
    .groupBy("month")
    .agg(sum("precipitation").alias("total_rainfall"))
    .sort("month")
    .show()
)


+-----+------------------+
|month|    total_rainfall|
+-----+------------------+
|    1|465.99999999999994|
|    2|             422.0|
|    3|             606.2|
|    4|             375.4|
|    5|             207.5|
|    6|             132.9|
|    7|              48.2|
|    8|             163.7|
|    9|235.49999999999997|
|   10|             503.4|
|   11|             642.5|
|   12| 622.7000000000002|
+-----+------------------+



In [79]:
(
    weather.filter(month("date") == 12)
    .filter(year("date") == 2013)
    .groupBy("weather")
    .agg(mean("temp_avg"))
    .show()
)

+-------+-----------------+
|weather|    avg(temp_avg)|
+-------+-----------------+
|    fog|7.555555555555555|
|    sun|2.977272727272727|
+-------+-----------------+



Here we first have a couple of .filter calls in order to restrict our data to December of 2013. We then group by the weather column, and lastly, aggregate by taking the average of our temp_avg column. The combination of group by and agg will calculate the average temperature for each unique value of the weather column.

Let's now find out how many days had freezing temperatures in each month of 2013.

In [80]:
(
    weather.filter(year("date") == 2013)
    .withColumn("freezing_temps", (weather.temp_avg <= 0).cast("int"))
    .withColumn("month", month("date"))
    .groupBy("month")
    .agg(sum("freezing_temps").alias("no_of_days_with_freezing_temps"))
    .sort("month")
    .show()
)


+-----+------------------------------+
|month|no_of_days_with_freezing_temps|
+-----+------------------------------+
|    1|                             3|
|    2|                             0|
|    3|                             0|
|    4|                             0|
|    5|                             0|
|    6|                             0|
|    7|                             0|
|    8|                             0|
|    9|                             0|
|   10|                             0|
|   11|                             0|
|   12|                             5|
+-----+------------------------------+



One last example, let's calculate the average temperature for each quarter of each year:

In [81]:
(
    weather.withColumn("quarter", quarter("date"))
    .withColumn("year", year("date"))
    .groupBy("year", "quarter")
    .agg(mean("temp_avg").alias("temp_avg"))
    .sort("year", "quarter")
    .show()
)


+----+-------+------------------+
|year|quarter|          temp_avg|
+----+-------+------------------+
|2012|      1| 5.587912087912088|
|2012|      2|12.675824175824175|
|2012|      3|            18.375|
|2012|      4| 8.581521739130435|
|2013|      1| 6.405555555555556|
|2013|      2|14.505494505494505|
|2013|      3| 19.47826086956522|
|2013|      4| 8.032608695652174|
|2014|      1| 7.205555555555556|
|2014|      2|14.296703296703297|
|2014|      3|19.858695652173914|
|2014|      4|  9.88586956521739|
|2015|      1| 8.972222222222221|
|2015|      2|15.258241758241759|
|2015|      3|19.407608695652176|
|2015|      4| 8.956521739130435|
+----+-------+------------------+



Here we create the quarter and year columns, then group by these two new columns, and take the average temperature as our aggregate. Lastly, we sort by the year and quarter for presentation purposes.

We could also use a pivot table like this:

In [82]:
(
    weather.withColumn("quarter", quarter("date"))
    .withColumn("year", year("date"))
    .groupBy("quarter")
    .pivot("year")
    .agg(expr("ROUND(MEAN(temp_avg), 2) AS temp_avg"))
    .sort("quarter")
    .show()
)


+-------+-----+-----+-----+-----+
|quarter| 2012| 2013| 2014| 2015|
+-------+-----+-----+-----+-----+
|      1| 5.59| 6.41| 7.21| 8.97|
|      2|12.68|14.51| 14.3|15.26|
|      3|18.38|19.48|19.86|19.41|
|      4| 8.58| 8.03| 9.89| 8.96|
+-------+-----+-----+-----+-----+



Here instead of grouping by two columns, we grouped by the first column and pivoted on the other column.

## Joins
Like pandas and sql, spark has functionality that lets us combine two tabular datasets, known as a join.

We'll start by creating some data that we can join together:

In [83]:
users = spark.createDataFrame(
    pd.DataFrame(
        {
            "id": [1, 2, 3, 4, 5, 6],
            "name": ["bob", "joe", "sally", "adam", "jane", "mike"],
            "role_id": [1, 2, 3, 3, np.nan, np.nan],
        }
    )
)
roles = spark.createDataFrame(
    pd.DataFrame(
        {
            "id": [1, 2, 3, 4],
            "name": ["admin", "author", "reviewer", "commenter"],
        }
    )
)
print("--- users ---")
users.show()
print("--- roles ---")
roles.show()


--- users ---
+---+-----+-------+
| id| name|role_id|
+---+-----+-------+
|  1|  bob|    1.0|
|  2|  joe|    2.0|
|  3|sally|    3.0|
|  4| adam|    3.0|
|  5| jane|    NaN|
|  6| mike|    NaN|
+---+-----+-------+

--- roles ---
+---+---------+
| id|     name|
+---+---------+
|  1|    admin|
|  2|   author|
|  3| reviewer|
|  4|commenter|
+---+---------+



To join two dataframes together, we'll need to call the .join method on one of them and supply the other as an argument. In addition, we'll need to supply the condition on which we are joining. In our case, we are joining where the role_id column on the users table is equal to the id column on the roles table.

In [84]:
users.join(roles, on=users.role_id == roles.id).show()


+---+-----+-------+---+--------+
| id| name|role_id| id|    name|
+---+-----+-------+---+--------+
|  1|  bob|    1.0|  1|   admin|
|  2|  joe|    2.0|  2|  author|
|  4| adam|    3.0|  3|reviewer|
|  3|sally|    3.0|  3|reviewer|
+---+-----+-------+---+--------+



By default, spark will perform an inner join, meaning that records from both dataframes will have a match with the other. We can also specify either a left or a right join, which will keep all of the records from either the left or right side, even if those records don't have a match with the other dataframe.

In [85]:
users.join(roles, on=users.role_id == roles.id, how="left").show()


+---+-----+-------+----+--------+
| id| name|role_id|  id|    name|
+---+-----+-------+----+--------+
|  1|  bob|    1.0|   1|   admin|
|  2|  joe|    2.0|   2|  author|
|  3|sally|    3.0|   3|reviewer|
|  4| adam|    3.0|   3|reviewer|
|  5| jane|    NaN|null|    null|
|  6| mike|    NaN|null|    null|
+---+-----+-------+----+--------+



In [86]:
users.join(roles, on=users.role_id == roles.id, how="right").show()


+----+-----+-------+---+---------+
|  id| name|role_id| id|     name|
+----+-----+-------+---+---------+
|   1|  bob|    1.0|  1|    admin|
|   2|  joe|    2.0|  2|   author|
|   4| adam|    3.0|  3| reviewer|
|   3|sally|    3.0|  3| reviewer|
|null| null|   null|  4|commenter|
+----+-----+-------+---+---------+



Notice that examples above have a duplicate id column. There are several ways we could go about dealing with this:

alias each dataframe + explicitly select columns after joining (this could also be implemented with spark SQL)
rename duplicated columns before merging
drop duplicated columns after the merge (.drop(right.id))
Visualization (or Lack Therof)
Spark does not provide a way to do visualization with their dataframes. To visualize data from spark, you should use the .toPandas method on a spark dataframe to convert it to a pandas dataframe, then visualize as you normally would.